# Progetto d'esame di Social Network Analysis
### Leonardo Bigelli
Sviluppo di una rete neurale per il rilevamento di attività coordinate all'interno di account presente in un Social Network.

### Import librerie necessarie per lo sviluppo

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import pandas as pd

### Caricamento dei dati simulati
Necessario un caricamento in locale

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
# I dati vengono simulati con nomi alfanumerici, quindi è necessaria una conversione per usarli come
# input per una rete neurale
df['account_id'] = df['account_id'].str.replace('account_', '', regex=False)
df['account_id'] = df['account_id'].astype(int)

df['account_id_y'] = df['account_id_y'].str.replace('account_', '', regex=False)
df['account_id_y'] = df['account_id_y'].astype(int)

df['object_id'] = df['object_id'].str.replace('object_', '', regex=False)
df['object_id'] = df['object_id'].astype(int)

df['content_id'] = df['content_id'].str.replace('share_', '', regex=False)
df['content_id'] = df['content_id'].astype(int)

df['content_id_y'] = df['content_id_y'].str.replace('share_', '', regex=False)
df['content_id_y'] = df['content_id_y'].astype(int)

df['time_delta'] = df['time_delta'].astype(int)

In [ ]:
try:
  x = df[['object_id', 'content_id','content_id_y','time_delta',
          'account_id', 'account_id_y']].values
  y = df['coordinated'].values
  x = np.array(x)
  y = np.array(y).astype(int)
  print("x shape:", x.shape)
  print("y shape:", y.shape)
except NameError:
  print("Errore: df non trovato")
except KeyError as e:
    print(f"Errore: colonna {e} non trovata")
except Exception as e:
  print(f"Errore: {e}")


x shape: (897039, 6)
y shape: (897039,)


In [ ]:
y = y.astype(int)

In [ ]:
# Suddivisione in train e test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Definizione e addestramento del modello

In [ ]:
# Definizione del modello per il riconoscimento di atività coordinate
model = tf.keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(x_train.shape[1],)),  # Primo layer nascosto
    layers.Dropout(0.2),
    layers.Dense(8, activation='relu'),  # Secondo layer nascosto
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # Output layer per classificazione binaria
])

# Compilazione del modello
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Funzione di perdita per problemi binari
              metrics=['accuracy'])

# Addestramento del modello
model.fit(x_train, y_train, epochs=20, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 57s 2ms/step - accuracy: 0.9846 - loss: 12.8501
Epoch 2/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 77s 2ms/step - accuracy: 0.9966 - loss: 0.0184
Epoch 3/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.9966 - loss: 0.0174
Epoch 4/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - accuracy: 0.9966 - loss: 0.0166
Epoch 5/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - accuracy: 0.9967 - loss: 0.0160
Epoch 6/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.9967 - loss: 0.0156
Epoch 7/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 51s 2ms/step - accuracy: 0.9967 - loss: 0.0159
Epoch 8/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 51s 2ms/step - accuracy: 0.9967 - loss: 0.0162
Epoch 9/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.9966 - loss: 0.0153
Epoch 10/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.9967 - loss: 0.0154
Epoch 11/20
22426/22426 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 0.9967 - loss: 0.0

In [ ]:
# Valutazione del modello
model.evaluate(x_test, y_test)

5607/5607 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9962 - loss: 0.0129


[0.011958499439060688, 0.9965330362319946]

### Salvataggio del modello su Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Salvataggio del modello sul Drive
model.save('/content/drive/MyDrive/Social_network_analysis/modello_coordinamento.keras')

In [ ]:
#test sul dataset che ha diverse time_window rispetto alla fase di training (nnecessario il caricamento)
model.evaluate(x, y)

14013/14013 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - accuracy: 0.9973 - loss: 0.0093


[0.010705000720918179, 0.9969046115875244]

## Caricamento del modello pre-addestrato, per valutazione su dati con diversi time_window
I datidevono essere caricati nella cella predisposta al caricamento. Si utilizza 'x' e 'y' non suddivisi in train e test in quanto non è necessario addestrare nuovamente il modello.

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Social_network_analysis/modello_coordinamento.keras')

In [ ]:
model.evaluate(x, y)

14017/14017 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9970 - loss: 0.0113


[0.01270474772900343, 0.9966422319412231]